
### Structured output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

### Pydantic
Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.

In [1]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model=init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000012FCD0E1010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000012FCD0E1D30>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
from pydantic import BaseModel,Field

# we inherit BaseModel
class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

The output of the LLM model should follow the above code's structure

In [3]:
model_with_structure=model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000012FCD0E1010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000012FCD0E1D30>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', '

#### Default output

In [4]:
model.invoke("Provide details about the moview Inception")

AIMessage(content='<think>\nOkay, I need to provide details about the movie Inception. Let me start by recalling what I know. Inception is a 2010 film directed by Christopher Nolan. It\'s a science fiction action movie. The main idea is about entering dreams to implant or extract ideas. The cast includes Leonardo DiCaprio as Dom Cobb, who\'s the main character. Other actors like Joseph Gordon-Levitt, Ellen Page, and Tom Hardy are also in it.\n\nThe plot revolves around a technique called "inception," which involves planting an idea into someone\'s subconscious. Cobb is a thief who does this, but he\'s being offered a chance to erase his criminal past by performing the opposite—extracting an idea. The movie deals with layers of dreams within dreams, different levels of reality, and the concept of time dilation in dreams. There\'s a lot of action, especially the famous rotating hallway fight scene by Joseph Gordon-Levitt.\n\nThemes include reality vs. dreams, the power of the subconsciou

#### Structured output 

In [5]:
response=model_with_structure.invoke("Provide details about the moview Inception")
response

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

### Message output alongside parsed structure

In [7]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

# this also will show the raw message
model_with_structure = model.with_structured_output(Movie, include_raw=True)  

response = model_with_structure.invoke("Provide details about the movie Inception")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking for details about the movie Inception. Let me recall what I know about that movie. Inception is a science fiction action film directed by Christopher Nolan. It was released in 2010. The movie is known for its complex plot involving dreams within dreams and features a star-studded cast including Leonardo DiCaprio. The rating is pretty high; I think it's around 8.8 on IMDb. So, I need to structure this information into the Movie function provided. The required parameters are title, year, director, and rating. Let me make sure I have all those details correct. The title is Inception, the year is definitely 2010, director is Christopher Nolan, and the rating is 8.8. That should cover all the required fields. Let me double-check the rating to be safe. Yes, IMDb does list it as 8.8. Alright, that's all set. Time to format that into the tool call.\n", 'tool_calls': [{'id': 'cwj149517', 'function': 

### Nested Structure

In [8]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str
    role: str

# we might have multiple actors so we use it in MovieDetails class (nested)
class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)

response = model_with_structure.invoke("Provide details about the movie Inception")
response

MovieDetails(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne'), Actor(name='Tom Hardy', role='Eames')], genres=['Science Fiction', 'Action', 'Thriller'], budget=None)

### TypedDict
TypedDict provides a simpler alternative using Python’s built-in typing, ideal when you don’t need runtime validation.

With TypedDict the output will be in the form of dictionary.

In [9]:
from typing_extensions import TypedDict,Annotated

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]


model_withtypedict=model.with_structured_output(MovieDict)
response=model_withtypedict.invoke("Please provide the details of the movie avengers")
response

{'director': 'Joss Whedon', 'rating': 8, 'title': 'The Avengers', 'year': 2012}

#### Nested Structure

In [10]:
class Actor(TypedDict):
    name: str
    role: str

class MovieDetails(TypedDict):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)

response = model_with_structure.invoke("Provide details about the movie Avengers")
response

{'budget': 220000000,
 'cast': [{'name': 'Robert Downey Jr.', 'role': 'Iron Man'},
  {'name': 'Chris Evans', 'role': 'Captain America'},
  {'name': 'Mark Ruffalo', 'role': 'Hulk'},
  {'name': 'Chris Hemsworth', 'role': 'Thor'},
  {'name': 'Scarlett Johansson', 'role': 'Black Widow'},
  {'name': 'Jeremy Renner', 'role': 'Hawkeye'}],
 'genres': ['Action', 'Sci-Fi', 'Adventure'],
 'title': 'Avengers',
 'year': 2012}

We can see the profile of the actual model, but not the structured model

In [ ]:
model.profile 

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

### DataClasses
A data class is a class typically containing mainly data, although there aren’t really any restrictions. You create it using the @dataclass decorator

In [12]:
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [13]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent


class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
    model="gpt-5",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result

{'messages': [HumanMessage(content='Extract contact info from: John Doe, john@example.com, (555) 123-4567', additional_kwargs={}, response_metadata={}, id='e28811f4-89d0-4cb4-b5f1-3550b01ec9cd'),
  AIMessage(content='{"name":"John Doe","email":"john@example.com","phone":"(555) 123-4567"}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 932, 'prompt_tokens': 204, 'total_tokens': 1136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 896, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqA2c3m4JqTiE8IX1IIiKMmHTLCjB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b4e72-8d56-7851-a333-649bfde0f3a5-0', usage_metadata={'input_tokens': 204, 'output_tokens': 932, 'total_tokens': 1136, 

In [14]:
result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [15]:
## Typedict
from typing_extensions import TypedDict
from langchain.agents import create_agent


class ContactInfo(TypedDict):
    """Contact information for a person."""
    name: str # The name of the person
    email: str # The email address of the person
    phone: str # The phone number of the person

agent = create_agent(
    model="gpt-5",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# {'name': 'John Doe', 'email': 'john@example.com', 'phone': '(555) 123-4567'}

{'name': 'John Doe', 'email': 'john@example.com', 'phone': '(555) 123-4567'}

In [16]:
## Dataclass

from dataclasses import dataclass
from langchain.agents import create_agent

# we don't need to inherit from the Base class
@dataclass
class ContactInfo:
    """Contact information for a person."""
    name: str # The name of the person
    email: str # The email address of the person
    phone: str # The phone number of the person


agent = create_agent(
    model="gpt-5",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')